In [67]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# อ่านไฟล์ CSV
file_path = 'sit-stand-walk-sleep.csv'  # กำหนดไฟล์ CSV
df = pd.read_csv(file_path)  # อ่านข้อมูลจากไฟล์ CSV

# แสดง DataFrame ในรูปแบบตาราง
df

In [ ]:
# ฟังก์ชันสำหรับการสร้าง skeleton พร้อม grid
def plot_skeleton(frame_data, frame_number):
    # กำหนดจุด landmark ที่จะเชื่อมต่อกัน
    connections = [
        (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8), (9, 10),  # หัว
        (11, 12), (12, 24), (24, 23), (23, 11),  # ลำตัว
        (11, 13), (13, 15), (12, 14), (14, 16), # แขน
        (15, 17), (15, 21), (15, 19), (19, 17), # มือซ้าย
        (16, 18), (16, 22), (16, 20), (20, 18), # มือขวา
        (23, 25), (25, 27), (27, 29), (29, 31), (31, 27),  # ขาขวา
        (24, 26), (26, 28), (28, 30), (30, 32), (28, 32)  # ขาซ้าย
    ]
    
    # ดึงค่าข้อมูล x และ y สำหรับแต่ละ landmark
    x_vals = [frame_data[f'x{i}'] for i in range(33)]
    y_vals = [frame_data[f'y{i}'] for i in range(33)]
    z_vals = [frame_data[f'z{i}'] for i in range(33)]
    
    # สร้างภาพแบบ subplot ที่มี 2D และ 3D
    fig = plt.figure(figsize=(15, 5))
    
    # วาด 2D skeleton ใน subplot แรก
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.set_title(f'2D Poses at Frame {frame_number}')
    ax1.scatter(x_vals, y_vals, c='green')  # วาดจุด
    for conn in connections:
        ax1.plot([x_vals[conn[0]], x_vals[conn[1]]], [y_vals[conn[0]], y_vals[conn[1]]], 'r-')  # วาดเส้นเชื่อมต่อ
    ax1.grid(True)  # เปิดกริด
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    
    # กำหนดขนาดสูงสุดของแกน
    ax1.set_xlim(0, 1920)  # ตั้งค่าขนาดแกน x
    ax1.set_ylim(1080, 0)  # ตั้งค่าขนาดแกน y

    # บันทึกภาพเป็นไฟล์ PNG    
    plt.savefig(f"output_frames/{frame_number}")


    # ************************************** กำลังแก้
    # วาด 3D skeleton ใน subplot ที่สอง
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    ax2.set_title(f'3D Poses at Frame {frame_number}')
    
    # ******* ตรงนี้ต้องกลับมาดูใหม่
    # ลำดับค่าพิกัด: 1 เป็นความลึก, 2 เป็นความสูง (invert), 3 เป็นแนวกว้าง
    ax2.scatter(x_vals, z_vals, y_vals, c='green')  # วาดจุด

    for conn in connections:
        ax2.plot([z_vals[conn[0]], z_vals[conn[1]]],  
                 [y_vals[conn[0]], y_vals[conn[1]]],  
                 [x_vals[conn[0]], x_vals[conn[1]]], 'r-') 
    
    ax2.set_xlabel('X')  
    ax2.set_ylabel('Y')  
    ax2.set_zlabel('Z') 
    ax2.grid(True)  # เปิดกริด

    # กำหนดขนาดสูงสุดของแกน
    ax2.set_xlim(0, 1920)  # ตั้งค่าขนาดแกน x
    ax2.set_ylim(1080, 0)  # ตั้งค่าขนาดแกน y
    ax2.set_zlim(0, 1)  # ตั้งค่าขนาดแกน y

    plt.show()

# แสดง skeleton สำหรับเฟรมที่เลือก
selected_frames = df['frames'].tolist()
# selected_frames = [200, 205, 210, 215, 230, 245, 260, 300, 450, 575, 675, 700, 725, 750, 845, 875, 900]

# แสดง skeleton
for frame in selected_frames:
    frame_data = df[df['frames'] == frame].iloc[0]
    plot_skeleton(frame_data, frame)

In [ ]:
# โฟลเดอร์สำหรับบันทึกภาพกราฟแต่ละเฟรม
output_dir = 'output_frames'
os.makedirs(output_dir, exist_ok=True)

# สร้างวิดีโอจากภาพที่บันทึกไว้โดยใช้ OpenCV
video_path = 'output_video.mp4'
frame_rate = 10  # กำหนด frame rate ของวิดีโอ
frame_size = (1920, 1080)  # ขนาดเฟรม (กว้าง x สูง)

# ใช้ OpenCV ในการสร้างวิดีโอ
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, frame_size)

# อ่านภาพที่บันทึกไว้และใส่ลงในวิดีโอ
for frame in selected_frames:
    img_path = os.path.join(output_dir, f'{frame}.png')
    img = cv2.imread(img_path)
    
    # ปรับขนาดภาพถ้าจำเป็น
    img = cv2.resize(img, frame_size)
    
    # ใส่ภาพเข้าไปในวิดีโอ
    out.write(img)

# ปิดการเขียนวิดีโอ
out.release()

print(f'Video saved as {video_path}')